### Кейс №2 - Приложение доставки

Вы работаете аналитиком в приложении по доставке продуктов. Команда внедрила в приложение умную систему рекомендации товаров – предполагается, что такая система поможет пользователям эффективнее работать с приложением и лучше находить необходимые товары.

Чтобы проверить эффективность системы рекомендаций, был проведен АБ-тест. В группе 1 оказались пользователи с новой системой рекомендаций, в группе 0 пользователи со старой версией приложения, где нет рекомендации товаров.

Ваша задача – оценить, смогла ли новая система рекомендаций принести пользу бизнесу и пользователям приложения. Для этого нужно выбрать метрики, которые отвечают за качество сервиса, и статистически сравнить эти метрики в двух группах.

Результат вашей работы – аналитическое заключение с ответом на вопрос, стоит ли включать новую систему рекомендаций на всех пользователей.

In [27]:
import pandas as pd 

import scipy.stats as ss

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
df_user_data = pd.read_csv('ab_users_data.csv')
df_order = pd.read_csv('ab_orders.csv')
df_products = pd.read_csv('ab_products.csv')

In [21]:
df_user_data.head()

,user_id,order_id,action,time,date,group
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0
1,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1
2,964,1257,create_order,2022-08-26 00:02:27,2022-08-26,0
3,966,1258,create_order,2022-08-26 00:02:56,2022-08-26,0
4,967,1259,create_order,2022-08-26 00:03:37,2022-08-26,1


In [20]:
df_user_data.dtypes

user_id              int64
order_id             int64
action              object
time        datetime64[ns]
date        datetime64[ns]
group                int64
dtype: object

In [19]:
df_user_data['time'] = pd.to_datetime(df_user_data['time'])
df_user_data['date'] = pd.to_datetime(df_user_data['date'], format='%Y-%m-%d')

In [9]:
df_user_data.shape

(4337, 6)

In [29]:
df_user_data.groupby('group')\
    .agg({'order_id':'count'})

,order_id
group,
0,1691
1,2646


In [35]:
df_order.head()

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19,"{75, 22, 53, 84}"
1,1256,2022-08-26 00:02:21,"{56, 76, 39}"
2,1257,2022-08-26 00:02:27,"{76, 34, 41, 38}"
3,1258,2022-08-26 00:02:56,"{74, 6}"
4,1259,2022-08-26 00:03:37,"{20, 45, 67, 26}"


In [34]:
df_order['creation_time'] = pd.to_datetime(df_order['creation_time'])

In [ ]:
def purchase_amount(x):
    id_product = list(map(int, x))
    for item in id_product:
        

In [ ]:
df_order[list_product_ids]

In [36]:
df_products.head()

,product_id,name,price
0,1,сахар,150.0
1,2,чай зеленый в пакетиках,50.0
2,3,вода негазированная,80.4
3,4,леденцы,45.5
4,5,кофе 3 в 1,15.0


In [37]:
df_products.dtypes

product_id      int64
name           object
price         float64
dtype: object

In [38]:
df_products.loc

In [41]:
?? df_order.loc